In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [3]:
data = pd.read_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset_filtered.csv')

data['date_field'] = pd.to_datetime(data['date_field'])

In [4]:

data = data[data['regional_master'] != 50000002]


In [5]:
data = data.set_index('date_field')

In [6]:
def fit_model(group):
    # Fit SARIMA model
    model = SARIMAX(group, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52))
    fitted_model = model.fit(disp=False)
    forecast = fitted_model.forecast(52)  # Forecast for the next 52 weeks
    return forecast


In [7]:
train_size = int(len(data) * 0.8) 
train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]

In [8]:
train_forecasts = []
for rm, group in train_data.groupby('regional_master')['sales']:
    forecast_data = fit_model(group)
    forecast_df = pd.DataFrame({'regional_master': rm, 'forecast': forecast_data})
    train_forecasts.append(forecast_df)

/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index i

In [9]:
train_forecast_df = pd.concat(train_forecasts)

In [10]:
import matplotlib.pyplot as plt

In [ ]:
for rm, group in train_forecast_df.groupby('regional_master'):
    actual = train_data.loc[train_data['regional_master'] == rm, 'sales'].values[-52:]
    forecast_values = group['forecast'].values[-52:]  # Using last 52 weeks of forecast
    
    plt.figure(figsize=(12, 6))
    plt.plot(actual, label='Actual', linestyle='-', marker='o')
    plt.plot(range(len(actual), len(actual) + len(forecast_values)), forecast_values, label='Forecast', linestyle='--')

    plt.xlabel('Week')
    plt.ylabel('Sales')
    plt.title(f'Actual vs Forecast Sales for {rm} (Training Data)')
    plt.legend()
    plt.grid(True)
    plt.show()


In [12]:
def smape(actual, forecast):
    """
    Calculate the Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    actual, forecast = np.array(actual), np.array(forecast)
    numerator = np.abs(forecast - actual)
    denominator = (np.abs(actual) + np.abs(forecast)) / 2
    smape_values = (numerator / denominator) * 100
    return np.mean(smape_values)

In [13]:
test_smape_scores = []
for rm, group in train_forecast_df.groupby('regional_master'):
    actual = test_data.loc[test_data['regional_master'] == rm, 'sales'].values[-52:]
    forecast_values = group['forecast'].values[-33:]  # Using last 52 weeks of forecast
    smape_score = smape(actual, forecast_values)
    if not np.isnan(smape_score):
        test_smape_scores.append(smape_score)

ValueError: operands could not be broadcast together with shapes (33,) (0,) 

In [ ]:
test_smape_scores = []
for rm, group in train_forecast_df.groupby('regional_master'):

    if rm in test_data['regional_master'].unique():
        actual = test_data.loc[test_data['regional_master'] == rm, 'sales'].values[-52:]
        forecast_values = group['forecast'].values[-33:]  # Using last 52 weeks of forecast
        smape_score = smape(actual, forecast_values)
        if not np.isnan(smape_score):
            test_smape_scores.append(smape_score)

In [ ]:
average_test_smape = np.sum(test_smape_scores) / len(test_smape_scores)
average_test_smape

21.09779775723815

In [ ]:
for rm, group in forecast_df.groupby('regional_master'):
    actual = data.loc[data['regional_master'] == rm, 'sales'].values[-52:]
    forecast_values = group['forecast'].values
    
    plt.figure(figsize=(12, 6))
    plt.plot(actual, label='Actual', linestyle='-', marker='o')
    plt.plot(range(len(actual), len(actual) + len(forecast_values)), forecast_values, label='Forecast', linestyle='--')

    plt.xlabel('Week')
    plt.ylabel('Sales')
    plt.title(f'Actual vs Forecast Sales for {rm}')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plt.xlabel('Week')
plt.ylabel('Sales')
plt.title('Actual vs Forecast Sales')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def smape(actual, forecast):
    """
    Calculate the Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    actual, forecast = np.array(actual), np.array(forecast)
    numerator = np.abs(forecast - actual)
    denominator = (np.abs(actual) + np.abs(forecast)) / 2
    smape_values = (numerator / denominator) * 100
    return np.mean(smape_values)

In [ ]:
smape_scores = []
for rm, group in forecast_df.groupby('regional_master'):
    actual = data.loc[data['regional_master'] == rm, 'sales'].values[-52:]
    forecast_values = group['forecast'].values
    smape_score = smape(actual, forecast_values)
    if not np.isnan(smape_score):
        smape_scores.append(smape_score)

In [ ]:
average_smape = np.sum(smape_scores) / data['regional_master'].nunique()
average_smape

41.995838924793865